 <span style="display:block;text-align:center;"><img src="https://i1.wp.com/bornsql.ca/wp-content/uploads/2018/10/logo-sql-715x400.png?fit=715%2C400&ssl=1"> <span style="display:block;text-align:center;"><img src="https://i.stack.imgur.com/qTvGA.png"></span></span> 
# <span style="color:#0080FF;">Created for: ARMA 4200</span>
## <span style="color:#0080FF;">Author: Kevin R. Warren, Principal Consultant Azure Cloud and AI, National Security Group, Microsoft Federal</span>
## <span style="color:#0080FF;">Author: Curtis Krumel, Senior Customer Engineer Azure Cloud and AI, National Security Group, Microsoft Federal</span>
<span style="font-size:9pt;">MICROSOFT MAKES NO WARRANTIES, EXPRESS OR IMPLIED, IN THIS DOCUMENT.</span> <span style="font-size:9.0pt;">Complying with all applicable copyright laws is the responsibility of the user.&nbsp; Without limiting the rights under copyright, no part of this document may be reproduced, stored in or introduced into a retrieval system, or transmitted in any form or by any means (electronic, mechanical, photocopying, recording, or otherwise), or for any purpose, without the express written permission of Microsoft Corporation.</span> <span style="font-size:9.0pt;">Microsoft may have patents, patent applications, trademarks, copyrights, or other intellectual property rights covering subject matter in this document.&nbsp; Except as expressly provided in any written license agreement from Microsoft, our provision of this document does not give you any license to these patents, trademarks, copyrights, or other intellectual property.</span> <span style="font-size:9.0pt;">The descriptions of other companies’ products in this document, if any, are provided only as a convenience to you.&nbsp; Any such references should not be considered an endorsement or support by Microsoft.&nbsp; Microsoft cannot guarantee their accuracy, and the products may change over time. Also, the descriptions are intended as brief highlights to aid understanding, rather than as thorough coverage. For authoritative descriptions of these products, please consult their respective manufacturers.</span> <span style="font-size:9.0pt;">© 2021 Microsoft Corporation. All rights reserved. Any use or distribution of these materials without express authorization of Microsoft Corp. is strictly prohibited.</span> <span style="font-size:9.0pt;">Microsoft and Windows are either registered trademarks or trademarks of Microsoft Corporation in the United States and/or other countries.</span> <span style="font-size:9.0pt;">The names of actual companies and products mentioned herein may be the trademarks of their respective owners.</span>

# <span style="color:#cc5500;">Introduction</span>

The purpose of this notebook is to provide database administrators (DBAs) with a clear and concise guide to the tasks and settings necessary to keep SQL Server 2016+ instances running safely and efficiently. The scripts and schedules in this guide are intended to be a starting place for maintenance. E ach organization and application is different, so changes in practice to the setting and schedules in this document are expected.

<span style="color:#5FB404;">At the top of your screen, in the menu bar above, select SQL as the Kernel, and Attach to: (select your SQL Instance)  When Prompted, Authenticate to your Instance and in the Drop-down, select your target database.</span>

## <span style="color:#cc5500;">Maintenance</span>

This section outlines for you, the tasks needed to operate and manage a Microsoft SQL Server. Further information on these tasks can be found in later sections of this document as linked.

### <span style="color:#cc5500;">Index Maintenance</span>

Index maintenance can be automated using the Maintenance Plan Wizard in SQL Server Management Studio.  Indexes in SQL Server can become fragmented over time which can affect performance. Reorganizing an index can reduce its fragmentation, can be interrupted at any time without a loss of work already performed, and can be run during production with almost no effect on performance. Rebuilding indexes will eliminate fragmentation in an index.  This process must complete in order to affect the performance of the database server.  The rebuild process is resource intensive, so rebuilding indexes should only be performed during maintenance windows.  It is common to reorganize indexes on a nightly or weekly basis and rebuild indexes on a weekly or monthly basis.  How often indexes should be reorganized and rebuilt depends on how quickly the indexes become fragmented.  The next section will describe one way to monitor for fragmentation which will help the DBA in deciding which indexes and databases should be maintained and the schedule for that maintenance.  You can read up on Index Maintenance here: [Maintenance Plans - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/maintenance-plans/maintenance-plans?view=sql-server-2017)

<span style="color:#5FB404;">Click in the Code block below, click the edit control and replace 'Adventureworks' with your database name, and 'HumanResources.Employee' with your table.</span>

<span style="color:#5FB404;">Then click the Run Cell button. To avoid repitition, it is assumed that you will modify the appropriate codeblocs for the remainder of this notebook</span>

In [ ]:
SELECT object_id, index_id, avg_fragmentation_in_percent, page_count
FROM sys.dm_db_index_physical_stats(DB_ID('AdventureWorks'), OBJECT_ID('HumanResources.Employee'), NULL, NULL, NULL);

The easiest way to maintain database indexes is to include daily reorganizations for all indexes and weekly rebuilding of indexes in a Maintenance Plan. Instructions for setting up a maintenance plan are located here: [Create a Maintenance Plan - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/maintenance-plans/create-a-maintenance-plan?view=sql-server-2017)

### <span style="color:#cc5500;">Statistics</span>

The Query Optimizer uses statistics to create query plans that improve query performance.  For most queries, the Query Optimizer already generates the necessary statistics for a high quality query plan; in some cases, you need to create additional statistics or modify the query design for best results.  Eabling Auto Update and Auto Create statistics alleviates most, if not all, of the need to manually create statistics.  As mentioned above, statistics are used by the Query Optimizer to help choose the best execution plan, including what indexes to use, when compiling a query.  When you perform index rebuild maintenance, any statistics associated with the index itself will be updated; however, any statistics associated with a column that is not included in an index will not be updated.  With Auto Update Statistics enabled and for SQL Server versions 2012 through 20142, the threshold for auto update of statistics requires 20% of rows in a table to change. Therefore, you should include an update statistics task in your maintenance plan. Updating statistics regularly can help further reduce the need for full index rebuild operations. A code Sample for updating Statistics will be provided below in the Configuration Section.  Brush up on your SQL Statistics Knowledge by reading this article: [Statistics - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/statistics/statistics?view=sql-server-2017) and [Default auto statistics update threshold change for SQL Server 2016 | Microsoft Docs](https://docs.microsoft.com/en-us/archive/blogs/psssql/default-auto-statistics-update-threshold-change-for-sql-server-2016)

## <span style="color:#cc5500;">Monitoring SQL Server</span>
This section outlines the tasks related to monitoring your SQL Server. Further information on these tasks can be found in later sections of this document as linked.
### <span style="color:#cc5500;">Review the SQL Server Error Log</span>
<span style="font-size:11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif;
color:#E3E3E3">View the SQL Server error log to ensure that processes have completed successfully (for example, backup and restore operations, batch commands, or other scripts and processes). This can be helpful to detect any current or potential problem areas, including automatic recovery messages (particularly if an instance of SQL Server has been stopped and restarted), kernel messages, or other server-level error messages.</span>
<span style="font-size:11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif;
color:#E3E3E3">View the SQL Server error log by using SQL Server Management Studio or any text editor. For more information about how to view the error log, see&nbsp;<a href="https://docs.microsoft.com/en-us/sql/relational-databases/logs/open-log-file-viewer?view=sql-server-ver15" data-linktype="relative-path" style="box-sizing: inherit;outline-color: inherit;
color:var(--theme-primary-base);cursor:pointer;overflow-wrap: break-word;
outline-style: initial;outline-width: 0px">Open Log File Viewer</a>. By default, the error log is located at&nbsp;<code style="box-sizing: inherit;
outline-color: inherit;background-color:var(--theme-inline-code);border-radius: 3px;
overflow-wrap: break-word"><span style="mso-ansi-font-size:11.0pt;mso-bidi-font-size:
11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif">Program Files\Microsoft SQL Server\MSSQL.</span></code><em style="box-sizing: inherit;outline-color: inherit">n</em><code style="box-sizing: inherit;
outline-color: inherit;background-color:var(--theme-inline-code);border-radius: 3px;
overflow-wrap: break-word"><span style="mso-ansi-font-size:11.0pt;mso-bidi-font-size:
11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif">\MSSQL\LOG\ERRORLOG</span></code>&nbsp;and&nbsp;<code style="box-sizing: inherit;outline-color: inherit;background-color:var(--theme-inline-code);
border-radius: 3px;overflow-wrap: break-word"><span style="mso-ansi-font-size:
11.0pt;mso-bidi-font-size:11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif">ERRORLOG.</span></code><em style="box-sizing: inherit;outline-color: inherit">n</em>&nbsp;files.</span>
<span style="font-size:11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif;
color:#E3E3E3">A new error log is created each time an instance of SQL Server is started, although the&nbsp;<a href="https://docs.microsoft.com/en-us/sql/relational-databases/system-stored-procedures/sp-cycle-errorlog-transact-sql?view=sql-server-ver15" data-linktype="relative-path" style="box-sizing: inherit;outline-color: inherit;
color:var(--theme-primary-base);cursor:pointer;overflow-wrap: break-word;
outline-style: initial;outline-width: 0px">sp_cycle_errorlog</a>&nbsp;system stored procedure can be used to cycle the error log files without having to restart the instance of SQL Server. Typically, SQL Server retains backups of the previous six logs and gives the most recent log backup the extension .1, the second most recent the extension .2, and so on. The current error log has no extension.</span>
<span style="font-size:11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif;
color:#E3E3E3">Be aware that you can also view the SQL Server error log on instances of SQL Server that are offline or cannot start. For more information, see&nbsp;<a href="https://docs.microsoft.com/en-us/sql/relational-databases/logs/view-offline-log-files?view=sql-server-ver15" data-linktype="relative-path" style="box-sizing: inherit;outline-color: inherit;
color:var(--theme-primary-base);cursor:pointer;overflow-wrap: break-word;
outline-style: initial;outline-width: 0px">View Offline Log Files</a>.</span>
### <span style="color:#cc5500;">Review the SQL Job History</span>
SQL Server utilizes the MSDB database in conjunction with the SQL Server Agent Service to execute and track jobs. These jobs perform functions such as executing maintenance plans described in the previous section Maintenance Guide. It is important to review the job history for success (or failure) of SQL Server jobs to ensure that maintenance and other automated tasks are functioning properly. The easiest way to review job history is by using SQL Server Management Studio:
1. In Object Explorer, connect to an instance of the SQL Server Database Engine, and then expand that instance.
2. Expand SQL Server Agent, and then expand Jobs.
3. Right-click a job, and then click View History.
4. In the Log File Viewer, view the job history.
5. To update the job history, click Refresh. To view fewer rows, click the Filter button and enter filter parameters

In [ ]:
-- lists all job information for the NightlyBackups job.  Mofidify this for your 'name'  
USE msdb ;  
GO  

EXEC dbo.sp_help_jobhistory   
    @job_name = N'NightlyBackups' ;  
GO

### <span style="color:#cc5500">Running DBCC CheckDB</span>

DBCC CHECKDB checks the logical and physical integrity of all objects in the specified database. All the DBCC validation commands use database snapshots to keep validation operations from interfering with on-going database operations and to allow the validation operation to see a quiescent, consistent view of the data. This snapshot can be as large as the database being checked, so make sure you have enough disk space available to accommodate the database snapshot

<span style="font-family:&quot;Segoe UI&quot;,sans-serif;
mso-fareast-font-family:&quot;Times New Roman&quot;;color:#E3E3E3">Checks the logical and physical integrity of all the objects in the specified database by performing the following operations:&lt;o:p&gt;</span>

- <span style="font-family:&quot;Segoe UI&quot;,sans-serif;
    mso-fareast-font-family:&quot;Times New Roman&quot;;color:#E3E3E3">Runs&nbsp;<a href="https://docs.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checkalloc-transact-sql?view=sql-server-2017"><span style="color:blue">DBCC CHECKALLOC</span></a>&nbsp;on the database.</span>
- Runs [<span style="color:blue">DBCC CHECKTABLE</span>](https://docs.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checktable-transact-sql?view=sql-server-2017) <span style="color: rgb(227, 227, 227); font-family: &quot;Segoe UI&quot;, sans-serif;">&nbsp;on every table and view in the database.</span>
- Runs [<span style="color:blue">DBCC CHECKCATALOG</span>](https://docs.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checkcatalog-transact-sql?view=sql-server-2017) <span style="color: rgb(227, 227, 227); font-family: &quot;Segoe UI&quot;, sans-serif;">&nbsp;on the database.</span>
- Validates the contents of every indexed view in the database.
- Validates link-level consistency between table metadata and file system directories and files when storing **varbinary(max)** <span style="color: rgb(227, 227, 227); font-family: &quot;Segoe UI&quot;, sans-serif;">&nbsp;data in the file system using FILESTREAM.</span>
- Validates the Service Broker data in the database.

Read up on DBCC here: [DBCC CHECKDB (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checkdb-transact-sql?view=sql-server-2017)

In [ ]:
-- Check the current database.    
DBCC CHECKDB;    
GO    
-- Check the AdventureWorks2012 database without nonclustered indexes.    
DBCC CHECKDB (AdventureWorks2012, NOINDEX);    
GO

The best way to run DBCC CheckDB on a consistent basis is to enable a step in a maintenance plan to check the consistency of the database. It is best to run the DBCC CheckDB maintenance task prior to performing the week’s full backup. Any consistency errors will be recorded in the maintenance task’s error log and help provide confidence in the integrity of the subsequent full database backup.

DBCC CheckDB should also be run after production databases are restored to test servers. This will test both the production database and the backup and restore system for errors,

<span style="color:#cc5500">Review the Cluster Log</span>

In clustered SQL Server environments, you might have availability problems or unexpected failovers causing you headaches. Factors external to SQL Server can contribute to these types of problems. Generating a set of cluster logs can help you quickly identify problems happening at the cluster level.

The following PowerShell command, executed from one of the nodes of your SQL Server cluster will produce a cluster log set in the directory of your choosing:

Here is a list of the PowerShell Cmlets related to the Cluster Log: [Get-ClusterLog (FailoverClusters) | Microsoft Docs](https://docs.microsoft.com/en-us/powershell/module/failoverclusters/get-clusterlog?view=windowsserver2019-ps&viewFallbackFrom=win10-ps)  Heres a good article on how to use PowerShell to find Cluster Log Errors: [Finding Cluster Log Errors with Powershell | John Morehouse (sqlrus.com)](https://sqlrus.com/2019/01/finding-cluster-log-errors/)

In [ ]:
## Change the Kernel from TSQL to PowerShell to run this
Get-ClusterLog -D C:\temp\clusterLogs